In [2]:
!pip install pyro-ppl

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: gosam 2.1.1-4b98559 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of gosam or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: professor2 X.Y.Z has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of professor2 or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [22]:

import sys
sys.path.append('/eos/home-i02/h/hsharma/.local/lib/python3.9/site-packages')

import torch
import pyro
import pyro.distributions as dist
from pyro.distributions.transforms import AffineAutoregressive, Permute
from pyro.distributions import TransformedDistribution
from pyro.nn import AutoRegressiveNN

# Sample 4-column dataset (make sure this is a torch.Tensor)
data = torch.tensor([
    [2.1, 1.2, 3.3, 4.4],
    [1.1, 2.3, 3.2, 4.1],
    [3.1, 0.9, 3.5, 4.6],
    [2.8, 1.7, 3.1, 4.9],
], dtype=torch.float32)

# Check the shape and type of data
print(f"Data shape: {data.shape}, Data type: {data.dtype}")

# Step 1: Define the base distribution (a simple multivariate normal distribution)
base_dist = dist.MultivariateNormal(torch.zeros(4), torch.eye(4))

# Step 2: Define an autoregressive neural network for the flow
arnn = AutoRegressiveNN(4, [16, 16])  # 4 input dimensions, with hidden layers of size 16

# Step 3: Define the Affine Autoregressive transforms (using the neural network)
transform1 = AffineAutoregressive(arnn)
transform2 = AffineAutoregressive(arnn)

# Step 4: Define the permutation to shuffle dimensions after each transformation
# Use a list of integers instead of a list of tensors for permutation
perm = Permute([3, 2, 1, 0])  # Permute the dimensions

# Step 5: Create a TransformedDistribution using multiple layers of transformations
flow_dist = TransformedDistribution(base_dist, [transform1, perm, transform2])

# Step 6: Collect the parameters of the learnable transforms (AffineAutoregressive)
params = list(transform1.parameters()) + list(transform2.parameters())

# Step 7: Optimizer to update the parameters
optimizer = torch.optim.Adam(params, lr=1e-3)
num_epochs = 1000

# Step 8: Training loop (simplified example)
for epoch in range(num_epochs):
    # Check the shape of data before passing to log_prob
    print(f"Epoch {epoch}: Input data shape: {data.shape}")

    try:
        # Log-likelihood of data
        log_prob = flow_dist.log_prob(data)

        # Check the output of log_prob
        print(f"Epoch {epoch}: Log probability: {log_prob}")

        # Loss is the negative log-likelihood
        loss = -log_prob.mean()  # Calculate mean loss

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if epoch % 100 == 0:
            print(f'Epoch {epoch}: Loss = {loss.item()}')
    except Exception as e:
        print(f"Error during epoch {epoch}: {str(e)}")

# Step 9: Sampling new data from the learned distribution
samples = flow_dist.sample((10,))
print("New samples from the learned distribution:")
print(samples)


Data shape: torch.Size([4, 4]), Data type: torch.float32
Epoch 0: Input data shape: torch.Size([4, 4])
Error during epoch 0: empty_like(): argument 'input' (position 1) must be Tensor, not list
Epoch 1: Input data shape: torch.Size([4, 4])
Error during epoch 1: empty_like(): argument 'input' (position 1) must be Tensor, not list
Epoch 2: Input data shape: torch.Size([4, 4])
Error during epoch 2: empty_like(): argument 'input' (position 1) must be Tensor, not list
Epoch 3: Input data shape: torch.Size([4, 4])
Error during epoch 3: empty_like(): argument 'input' (position 1) must be Tensor, not list
Epoch 4: Input data shape: torch.Size([4, 4])
Error during epoch 4: empty_like(): argument 'input' (position 1) must be Tensor, not list
Epoch 5: Input data shape: torch.Size([4, 4])
Error during epoch 5: empty_like(): argument 'input' (position 1) must be Tensor, not list
Epoch 6: Input data shape: torch.Size([4, 4])
Error during epoch 6: empty_like(): argument 'input' (position 1) must be T

TypeError: index_select() received an invalid combination of arguments - got (int, list), but expected one of:
 * (int dim, Tensor index)
      didn't match because some of the arguments have invalid types: (int, !list of [int, int, int, int]!)
 * (name dim, Tensor index)
      didn't match because some of the arguments have invalid types: (!int!, !list of [int, int, int, int]!)
